# Дистрибутивный анализ *то есть*

In [45]:
import openpyxl
import re
import pymystem3
from nltk.tokenize import word_tokenize
from pymystem3 import Mystem
import gensim
import logging
import nltk.data
import pandas as pd
import urllib.request
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from gensim.models import word2vec
import warnings
warnings.filterwarnings('ignore')
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
%matplotlib inline
from nltk.corpus import stopwords

## Работа с таблицей по векам

In [46]:
wb = openpyxl.load_workbook('/Users/annaaksenova/Desktop/Работа к Кувшинской/То есть выборка по векам.xlsx')
sheet = wb['21'] # Имя листа

In [47]:
m = Mystem()

In [48]:
v1 = []
# Собираем примеры из таблицы
for v in sheet.iter_rows(min_row=2, min_col=21, max_col=21, max_row=1001, values_only=True):
    if v[0] == None:
        v1.append('')
    else:
        v1.append(v[0].replace(u'\xa0', ' '))
v2 = []
for v in sheet.iter_rows(min_row=2, min_col=22, max_col=22, max_row=1001, values_only=True):
    if v[0] == None:
        v2.append('')
    else:
        item = v[0].replace(u'\xa0', ' ')
        item = re.sub('\[.+?\]', '', item)
        item = re.sub('[Тт]о есть', 'тоесть', item) # \b убирает случаи что есть
        v2.append(item)
values = [v1[i]+v2[i] for i in range(1000)]
print(values[:10])

[' Я засунула пластинку и блюдо в один пакет, положила под сиденье и ― надо знать наши самолёты советских времён, лишённые кондиционеров, ― в конце пути в Ереван достала из-под сиденья пластинку, принявшую форму таза, тоесть блюда.   ', ' Само название места происходит от того, что здесь врачевали «косимых» ― тоесть увечных.   ', ' Я даже не могу найти концов, тоесть понять, кто же мне, собственно, должен платить.   ', 'Никаких провокаций ― известные имена-брэнды.  Расчёт на то, что, даже если зритель не шибко приобщён к балету и не знает, кто такая Аньес Летестю, он непременно отреагирует на Opera National de Paris.  Или хотя бы на «Лебединое озеро». Или хотя бы на Paris.  Плюс гарантия качества: звёзды подобраны заслуженные, но при этом, что важно, действующие: спешно доставленные на пароходе прямо из Парижа, поднимешь крышку ― пар! тоесть готовые доказывать свой статус каждым спектаклем.   ', '  «Чтобы не быть выдавленными с рынка, ― говорит академик Алексей Макаров, ― необходимы же

In [49]:
tagged_corp = []
for sentence in values:
    ana = m.analyze(sentence)
    tagged_corp.append(ana)

In [50]:
print(len(tagged_corp))

1000


In [51]:
final = []
count = 1
for sent in tagged_corp:
    sentence = []
    for word in sent:
        if 'analysis' in word and word['analysis'] != []:
            gr = word['analysis'][0]['gr']
            pos = gr.split('=')[0].split(',')[0]
            lex = word['analysis'][0]['lex']
            token = word['text']
            if pos == 'V' or pos == 'S' or pos == 'A' or pos == 'ADV':
                sentence.append((token, lex, pos))
    if sentence == []:
        print(sent)
        print(count)
    else:
        final.append(sentence)
    count+=1

In [52]:
with open('sentences.txt', 'w') as fw:
    for sentence in final:
        for word in sentence:
            for part in word:
                fw.write(part + ' ')
        fw.write('\n')

In [53]:
sentence_counts = []
for sentence in final:
    left_sum = 0
    right_sum = 0
    count_l = 0
    count_r = 0
    ind = 0 # индикатор было ли вхождение то есть
    for word in sentence:
        if word[0] == 'тоесть':
            ind = 1
        if not ind:
            count_l += 1
            left_sum += ?? # тут вектор слова
        elif ind:
            count_r += 1
            left_sum += ?? # тут вектор слова
        
    count = (left_sum, right_sum)
    sentence_counts.append(count)

SyntaxError: invalid syntax (<ipython-input-53-03e136ed08c6>, line 13)

## Сравнение средних векторов правого и левого контекстов

Здесь я брала именно не сумму, а среднее, потому что количество слов разное

In [54]:
import re
part_sent = []

# тут я немного криво сохранила текстовый файл с сервера, просто считываю данные номер предложения + косинус
# при этом я выкинула примеры, где вектор хотябы одного контекста nan
with open('part_sim.txt') as fh: 
    lines = fh.readlines()
for line in lines:
    line = line.strip()
    a = re.search(r'([0-9]+)(-?0\.[0-9]+)', line)
    number = a.group(1)
    cos = a.group(2)
    part_sent.append([number, cos])


In [55]:
wb = openpyxl.load_workbook('/Users/annaaksenova/Desktop/Работа к Кувшинской/То есть выборка по векам.xlsx')
sheet = wb['21']
for i in part_sent: # для каждой пары номер-косинус вытаскиваем из таблицы разметку
    i.append(sheet['Q{}'.format(str(int(i[0])+1))].value)
    i.append(sheet['P{}'.format(str(int(i[0])+1))].value)

In [56]:
from collections import Counter

Сколько примеров каждого типа имеют схожесть контекста > 0.6

In [57]:
tags_near = [i[2] for i in part_sent if float(i[1]) > 0.6]
tags_near_common = Counter(tags_near)
print(len(tags_near))
tags_near_common

141


Counter({'определение': 12,
         'следствие': 49,
         'дополнение': 14,
         'спецификация': 9,
         'обобщение': 6,
         'переспрос инт': 5,
         'интерпретация': 22,
         'переформулирование': 17,
         'коррекция': 4,
         'пример': 2,
         'переспрос как': 1})

Сколько примеров каждого типа имеют схожесть контекста < 0.4

In [58]:
tags_far = [i[2] for i in part_sent if float(i[1]) < 0.4]
tags_far_common = Counter(tags_far)
print(len(tags_far))
tags_far_common

504


Counter({'переформулирование': 62,
         'интерпретация': 132,
         'обобщение': 23,
         'следствие': 110,
         'переспрос инт': 27,
         'дополнение': 40,
         'определение': 35,
         'спецификация': 30,
         'переспрос': 5,
         'коррекция': 18,
         'обр определение': 3,
         'дискурс': 7,
         'переспрос как': 2,
         'пояснение': 5,
         'модальность': 1,
         'пример': 4})

In [59]:
part_sent[:10]

[['292', '0.89379615', 'определение', 1],
 ['953', '0.8557891', 'следствие', 2],
 ['215', '0.8420359', 'следствие', 2],
 ['301', '0.8367445', 'дополнение', 2],
 ['557', '0.8128834', 'следствие', 2],
 ['505', '0.81279266', 'спецификация', 2],
 ['598', '0.80335826', 'определение', 1],
 ['62', '0.79922545', 'обобщение', 2],
 ['213', '0.7939003', 'переспрос инт', 2],
 ['826', '0.78499955', 'интерпретация', 1]]

Считаем медианы близости для каждого типа значений

In [64]:
import numpy as np

In [67]:
medians = {}
for x in part_sent:
    if x[2] not in medians.keys():
        medians[x[2]] = []
    medians[x[2]].append(float(x[1]))
for meaning, sim in medians.items():
    medians[meaning] = np.median(sim)

In [72]:
medians

{'определение': 0.43211296,
 'следствие': 0.4321039,
 'дополнение': 0.39632887,
 'спецификация': 0.36558703000000004,
 'обобщение': 0.419430125,
 'переспрос инт': 0.38228701,
 'интерпретация': 0.37420303,
 'переформулирование': 0.3672477,
 'коррекция': 0.41246367,
 'пример': 0.36468351,
 'переспрос как': 0.49007556,
 'дискурс': 0.232689005,
 'пояснение': 0.281846495,
 'обр определение': 0.33045346,
 'модальность': 0.395722895,
 'переспрос': 0.2415034}

## Медианы по самым похожим словам

Здесь я проделала то же самое, только выбрала самые похожие слова. Я работала с векторами на сервере, а потом выкачала только близость

In [79]:
with open('word_sim.txt') as fh:
    cos = [[float(i.strip())] for i in fh.readlines()]

In [80]:
wb = openpyxl.load_workbook('/Users/annaaksenova/Desktop/Работа к Кувшинской/То есть выборка по векам.xlsx')
sheet = wb['21']
for i in range(len(cos)): # для каждой пары номер-косинус вытаскиваем из таблицы разметку
    cos[i].append(sheet['Q{}'.format(str(i+2))].value)

In [81]:
medians_words = {}
for x in cos:
    if x[1] not in medians_words.keys():
        medians_words[x[1]] = []
    medians_words[x[1]].append(float(x[0]))
for meaning, sim in medians_words.items():
    medians_words[meaning] = np.median(sim)

In [82]:
medians_words

{'коррекция': 1.0,
 'определение': 0.5928247,
 'интерпретация': 0.5555712,
 'следствие': 0.67190266,
 'дополнение': 0.64230895,
 'переспрос как': 0.28707352,
 'переформулирование': 0.5521171,
 'переспрос инт': 0.59454864,
 'спецификация': 0.55591154,
 'пример': 0.487336125,
 'модальность': 1.0000001,
 'обобщение': 0.61480975,
 'пояснение': 0.7784355000000001,
 'дискурс': 0.327046635,
 'переспрос': 0.33812416,
 'обр определение': 0.3905912}

Смотрите, у коррекции медиана 1, потому что она обычно включает повтор "домой, то есть не домой, а в общежитие"

Я прямо совсе не понимаю, что значит 1.0000001 у модальности